# FAKE NEWS CLASSIFIER
## a machine learning approach towards fake news detection

DATA COLLECTION - WEB SCRAPING OF NEWS ARTICLES
- Legitimate newspaper websites: NYTimes, CNN, Market Watch, CNBC, The Guardian, Fox News etc.
- Fake news websites: Dailymash, The Onion, News Thump, Clickhole, 100percentfedup, Politifact etc.

In [2]:
pip install feedparser

     |████████████████████████████████| 194kB 6.5MB/s eta 0:00:01
  Created wheel for feedparser: filename=feedparser-5.2.1-cp37-none-any.whl size=44940 sha256=fd8a4023962cfb1162cd7593a20d07054242d395d04fb953104bb5fbf4fef86c
  Stored in directory: /Users/Liuyang/Library/Caches/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install newspaper3k

     |████████████████████████████████| 215kB 8.3MB/s eta 0:00:01
     |████████████████████████████████| 7.4MB 9.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 12.1MB/s eta 0:00:01
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=32e878b621c7fe993b236029b997ad509709e35c40a59a63277552b49063e39d
  Stored in directory: /Users/Liuyang/Library/Caches/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398405 sha256=e39b07cc225a43dfc57f4911c027cd43bb7cd316dac2aa6cd6727f44d38b6600
  Stored in directory: /Users/Liuyang/Library/Caches/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13538 sha256=433a2e0f3e1b2ed1210eec0dcbeefbda653772f7c995ef67a5d69f425c653662
  Stored in directory: /Users/Liuyang/Library/Caches/pip/wheels/81/2b/43/a0

In [11]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

In [12]:
# Set the limit for number of articles to download
LIMIT = 20000

In [13]:
data = {}
data['newspapers'] = {}

In [16]:
# Loads the JSON files with news sites
with open('data/NewsPapers.json') as data_file:
    companies = json.load(data_file)

In [17]:
count = 1

# Iterate through each news company
# the company is the name, the value is the dictionary of links
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                article['author'] = content.authors
                newsPaper['articles'].append(article)
                if count % 10 == 0:
                    print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 10:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            article['author'] = content.authors
            newsPaper['articles'].append(article)
            if count % 10 == 0: 
                print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper


# Finally it saves the articles as a JSON-file.
try:
    fname = 'scraped_articles.json'
    print('saving articles . . . in {}'.format(fname))
    with open(fname, 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)

10 articles downloaded from newyorktimes_business , url:  https://www.nytimes.com/2020/01/18/business/davos-china.html?emc=rss&partner=rss
20 articles downloaded from newyorktimes_business , url:  https://www.nytimes.com/2020/01/17/business/stock-market-rally.html?emc=rss&partner=rss
30 articles downloaded from newyorktimes_business , url:  https://www.nytimes.com/2020/01/17/business/books-debt-personal-finance.html?emc=rss&partner=rss
40 articles downloaded from newyorktimes_business , url:  https://www.nytimes.com/2020/01/17/business/bond-market-investments.html?emc=rss&partner=rss
50 articles downloaded from newyorktimes_business , url:  https://www.nytimes.com/2020/01/16/business/student-loans-forgiveness-taxes.html?emc=rss&partner=rss
10 articles downloaded from newyorktimes_science , url:  https://www.nytimes.com/2020/01/19/us/space-force-uniform-camo.html?emc=rss&partner=rss
20 articles downloaded from newyorktimes_science , url:  https://www.nytimes.com/2020/01/16/arts/worlds-b

Building prefix dict from /Users/Liuyang/opt/anaconda3/envs/withnewpkg/lib/python3.7/site-packages/jieba/dict.txt ...
Dumping model to file cache /var/folders/d4/ym508p7j2sx7n8kq2d37gzxm0000gn/T/jieba.cache
Loading model cost 1.7148289680480957 seconds.
Prefix dict has been built succesfully.


70 articles downloaded from newyorktimes_politics  using newspaper, url:  https://cn.nytimes.com/china/20200117/china-iran-us-weibo/
80 articles downloaded from newyorktimes_politics  using newspaper, url:  https://cn.nytimes.com/opinion/20200117/ai-weiwei-germany-china/
89  Article has date of type None...
90 articles downloaded from newyorktimes_politics  using newspaper, url:  https://cn.nytimes.com/culture/20200115/star-wars-china/
100  Article has date of type None...
101  Article has date of type None...
102  Article has date of type None...
103  Article has date of type None...
104  Article has date of type None...
105  Article has date of type None...
106  Article has date of type None...
107  Article has date of type None...
108  Article has date of type None...
109  Article has date of type None...
110  Article has date of type None...
Too many noneType dates, aborting...
Building site for  newyorktimes_world
10 articles downloaded from newyorktimes_world  using newspaper, ur